# We Are How You Mean
## Important, real stats that matter

In [1]:
import pandas as pd
import csv
import os

## Managers and Team Names

In [2]:
# teams list
teams = {"Tim" : "Violent Tyreke Roadshow",
         "Scott" : "MT KilledbyLamaro",
         "Johnny" : "2 Brothers in a Chiefs Van",
         "JT" : "We Are Climbing High",
         "Matt" : "Mike Evans down your Vincents",
         "Tyler" : "Pirates of the Pancreas",
         "Brian" : "MuscularMoore'sTurbulentJuice",
         "Jason" : "footBall Fondlers",
         "Kevin" : "Three unmuscular Michaels jr.",
         "David" : "Ants In My Eyes Jackson"}

## Load and Update Scores and Matchups
### Functions for Scores

In [3]:
# Initialize weekly scores
def initialize_scores():
    weekly_scores = {manager: {f"Week {week}": 0 for week in range(1, 18)} for manager in teams}
    return weekly_scores

# Save scores to CSV
def save_scores_to_csv(weekly_scores, filename='weekly_scores.csv'):
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        header = ["Manager"] + [f"Week {week}" for week in range(1, 18)]
        writer.writerow(header)
        
        for manager, scores in weekly_scores.items():
            row = [manager] + [scores[f"Week {week}"] for week in range(1, 18)]
            writer.writerow(row)

# Load scores from CSV
def load_scores_from_csv(filename='weekly_scores.csv'):
    if not os.path.exists(filename):
        return initialize_scores()
    
    weekly_scores = {}
    with open(filename, mode='r') as file:
        reader = csv.reader(file)
        next(reader)  # Skip the header
        for row in reader:
            manager = row[0]
            scores = {f"Week {i+1}": float(row[i+1]) if row[i+1] else 0 for i in range(17)}
            weekly_scores[manager] = scores
    return weekly_scores

# Update scores for a specific week
def update_weekly_scores(weekly_scores):
    # Ask for the week number
    while True:
        try:
            week = int(input("What week do you want to update? (1-17): "))
            if 1 <= week <= 17:
                break
            else:
                print("Please enter a valid week number between 1 and 17.")
        except ValueError:
            print("Please enter a valid number.")
    
    # Loop through each manager and update the score
    for manager in weekly_scores:
        try:
            score = float(input(f"What did {manager} score in Week {week}? "))
            weekly_scores[manager][f"Week {week}"] = score
        except ValueError:
            print("Please enter a valid score (numeric).")

    # Save updated scores to CSV
    save_scores_to_csv(weekly_scores)
    print(f"Scores for Week {week} updated successfully and saved.")

# Function to update an individual score
def update_individual_score(weekly_scores):
    # Ask for the manager's name
    manager = input("Which manager's score do you want to update? ").strip()
    
    if manager not in weekly_scores:
        print("Manager not found. Please make sure the name is correct.")
        return
    
    # Ask for the week number
    while True:
        try:
            week = int(input(f"What week do you want to update for {manager}? (1-17): "))
            if 1 <= week <= 17:
                break
            else:
                print("Please enter a valid week number between 1 and 17.")
        except ValueError:
            print("Please enter a valid number.")
    
    # Update the score
    try:
        score = float(input(f"What did {manager} score in Week {week}? "))
        weekly_scores[manager][f"Week {week}"] = score
        save_scores_to_csv(weekly_scores)
        print(f"Score for {manager} in Week {week} updated successfully and saved.")
    except ValueError:
        print("Please enter a valid score (numeric).")

### Functions for Matchups

In [4]:
# Initialize the weekly matchups
def initialize_matchups():
    weekly_matchups = {manager: {f"Week {week}": None for week in range(1, 18)} for manager in teams}
    return weekly_matchups

# Save matchups to CSV
def save_matchups_to_csv(weekly_matchups, filename='weekly_matchups.csv'):
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        header = ["Manager"] + [f"Week {week}" for week in range(1, 18)]
        writer.writerow(header)
        
        for manager, matchups in weekly_matchups.items():
            row = [manager] + [matchups[f"Week {week}"] if matchups[f"Week {week}"] else "" for week in range(1, 18)]
            writer.writerow(row)

# Load matchups from CSV
def load_matchups_from_csv(filename='weekly_matchups.csv'):
    if not os.path.exists(filename):
        return initialize_matchups()
    
    weekly_matchups = {}
    with open(filename, mode='r') as file:
        reader = csv.reader(file)
        next(reader)  # Skip the header
        for row in reader:
            manager = row[0]
            matchups = {f"Week {i+1}": row[i+1] if row[i+1] else None for i in range(17)}
            weekly_matchups[manager] = matchups
    return weekly_matchups

# Update matchups for a specific week
def update_weekly_matchups(weekly_matchups):
    # Ask for the week number
    while True:
        try:
            week = int(input("What week do you want to update matchups for? (1-17): "))
            if 1 <= week <= 17:
                break
            else:
                print("Please enter a valid week number between 1 and 17.")
        except ValueError:
            print("Please enter a valid number.")
    
    # Check if any matchups are already set for the chosen week
    already_set = any(weekly_matchups[manager][f"Week {week}"] is not None for manager in weekly_matchups)
    
    # Ask if user wants to reset if matchups already exist
    if already_set:
        reset = input(f"Matchups for Week {week} already exist. Do you want to reset them and start over? (yes/no): ").strip().lower()
        if reset != 'yes':
            print("No changes made. Exiting update process.")
            return

    # Reset all matchups for the specified week
    for manager in weekly_matchups:
        weekly_matchups[manager][f"Week {week}"] = None

    # Loop through each manager to set new matchups
    processed_managers = set()  # Keep track of managers we've already processed

    for manager in weekly_matchups:
        if manager in processed_managers:
            continue
        
        # Ask who this manager played
        while True:
            opponent = input(f"Who did {manager} play in Week {week}? ").strip()

            # Validate opponent exists, is not the same as the manager, and has not been processed yet
            if opponent not in teams or opponent == manager:
                print(f"{opponent} is not a recognized or valid opponent. Please try again.")
                continue
            
            # Check if opponent has already been processed
            if opponent in processed_managers:
                print(f"{opponent} has already been assigned a matchup. Please choose another opponent.")
                continue

            # If valid, update matchups symmetrically and mark both as processed
            weekly_matchups[manager][f"Week {week}"] = opponent
            weekly_matchups[opponent][f"Week {week}"] = manager
            processed_managers.update([manager, opponent])
            break

    # Save updated matchups to CSV
    save_matchups_to_csv(weekly_matchups)
    print(f"Matchups for Week {week} have been reset and updated successfully.")


# Function to update an individual matchup
def update_individual_matchup(weekly_matchups):
    # Ask for the manager's name
    manager = input("Which manager's matchup do you want to update? ").strip()
    
    if manager not in weekly_matchups:
        print("Manager not found. Please make sure the name is correct.")
        return
    
    # Ask for the week number
    while True:
        try:
            week = int(input(f"What week do you want to update for {manager}? (1-17): "))
            if 1 <= week <= 17:
                break
            else:
                print("Please enter a valid week number between 1 and 17.")
        except ValueError:
            print("Please enter a valid number.")
    
    # Ask for the opponent
    opponent = input(f"Who did {manager} play in Week {week}? ").strip()
    
    if opponent not in weekly_matchups:
        print(f"{opponent} is not a recognized manager. Please try again.")
        return
    
    # Update the matchup symmetrically
    weekly_matchups[manager][f"Week {week}"] = opponent
    weekly_matchups[opponent][f"Week {week}"] = manager
    save_matchups_to_csv(weekly_matchups)
    print(f"Matchup for {manager} in Week {week} updated successfully and saved.")

### Load existing stats

In [5]:
# Initialize or load the weekly scores
weekly_scores = load_scores_from_csv()

In [6]:
# Initialize or load the weekly matchups
weekly_matchups = load_matchups_from_csv()

### Update Scores

In [7]:
# # To update scores, call:
# update_weekly_scores(weekly_scores)

In [8]:
# # To correct or update and individual score
# update_individual_score(weekly_scores)

In [9]:
# Convert to df
weekly_scores_df = pd.DataFrame.from_dict(weekly_scores, orient='index')

# The columns should already be named as "Week 1", "Week 2", etc., but this ensures consistency
weekly_scores_df.columns = [f"Week {i+1}" for i in range(len(weekly_scores_df.columns))]

# # Display the DataFrame to confirm
# print(weekly_scores_df)

### Update Matchups

In [10]:
# # To update matchups for all managers in a specific week
# update_weekly_matchups(weekly_matchups)

In [11]:
# # To update an individual manager's matchup for a specific week
# update_individual_matchup(weekly_matchups)

In [12]:
weekly_matchups_df = pd.DataFrame.from_dict(weekly_matchups, orient='index')

# Ensure the columns are named consistently (e.g., "Week 1", "Week 2", ...)
weekly_matchups_df.columns = [f"Week {i+1}" for i in range(len(weekly_matchups_df.columns))]

# # Display the DataFrame to confirm
# print(weekly_matchups_df)

## What Week?

In [13]:
def determine_most_recent_week(weekly_scores_df):
    # Loop through the columns (weeks) to find the first week with a score of 0
    for week in weekly_scores_df.columns:
        # Check if any team has a score of 0 for this week
        if (weekly_scores_df[week] == 0).any():
            # Return the index of the most recent week before unplayed weeks
            most_recent_week = int(week.split()[1]) - 1
            print(f"The most recent week that has been played is Week {most_recent_week}.")
            return most_recent_week
    
    # If all weeks have been played (no zeros found), return the last week
    most_recent_week = len(weekly_scores_df.columns)
    print(f"All weeks have been played. The most recent week is Week {most_recent_week}.")
    return most_recent_week

In [14]:
most_recent_week = determine_most_recent_week(weekly_scores_df)

The most recent week that has been played is Week 6.


## Basic statistics

#### Standings, Average, Average range, Best Score, Worst Score

In [15]:
# Filter the DataFrame to include only played weeks
played_scores_df = weekly_scores_df.iloc[:, :most_recent_week]
played_matchups_df = weekly_matchups_df.iloc[:, :most_recent_week]

analysis_df = pd.DataFrame(index=played_scores_df.index)

## Determine Standings

# Step 1: Initialize Wins and Losses Tally
wins = {manager: 0 for manager in played_scores_df.index}
losses = {manager: 0 for manager in played_scores_df.index}

# Step 2: Calculate Wins and Losses
for week in played_scores_df.columns:
    for manager in played_scores_df.index:
        opponent = played_matchups_df.at[manager, week]
        
        # Only proceed if an opponent is listed
        if opponent and opponent in played_scores_df.index:
            manager_score = played_scores_df.at[manager, week]
            opponent_score = played_scores_df.at[opponent, week]
            
            # Determine who won and update tally
            if manager_score > opponent_score:
                wins[manager] += 1
            elif manager_score < opponent_score:
                losses[manager] += 1

# Step 3: Add Standings Column to analysis_df
analysis_df["Standings"] = [f"{wins[manager]} - {losses[manager]}" for manager in analysis_df.index]

In [16]:
## Calculate Total Points, Average Points, Highest, Lowest, and Average Range

# Total Points
analysis_df["Total Points"] = played_scores_df.sum(axis=1)

# Average Points
analysis_df["Average Points"] = played_scores_df.mean(axis=1).round(2)

# Standard Deviation
std_dev = played_scores_df.std(axis=1)

# # Average Range: (avg - std_dev) to (avg + std_dev)
# lower_bound = analysis_df["Average Points"] - std_dev
# upper_bound = analysis_df["Average Points"] + std_dev
# analysis_df["Average Range"] = lower_bound.round(2).astype(str) + " - " + upper_bound.round(2).astype(str)

# Highest Score
analysis_df["Highest Score"] = played_scores_df.max(axis=1)

# Lowest Score
analysis_df["Lowest Score"] = played_scores_df.min(axis=1)

# Count Times Each Manager Was the Highest Scorer ("Eyehole Man") and Lowest Scorer ("Fascist of the Week")
# Initialize counters
analysis_df["Eyehole Man"] = 0
analysis_df["Fascist of the Week"] = 0

# Loop through each week to find highest and lowest scorers
for week in played_scores_df.columns:
    # Find the highest and lowest scores for the week
    highest_scorer = played_scores_df[week].idxmax()
    lowest_scorer = played_scores_df[week].idxmin()
    
    # Increment counters
    analysis_df.at[highest_scorer, "Eyehole Man"] += 1
    analysis_df.at[lowest_scorer, "Fascist of the Week"] += 1

# Add a "Wins" column for sorting purposes
analysis_df["Wins"] = [wins[manager] for manager in analysis_df.index]

# Sort DataFrame by Standings
analysis_df = analysis_df.sort_values(by=["Wins", "Total Points"], ascending=[False, False])

# Drop the temporary "Wins" column (if you don't want it displayed)
analysis_df = analysis_df.drop(columns=["Wins"])

# Display the final analysis DataFrame
print(analysis_df)

       Standings  Total Points  Average Points  Highest Score  Lowest Score  \
Johnny     6 - 0        641.38          106.90         131.50         75.94   
Matt       5 - 1        610.84          101.81         131.38         82.68   
Brian      5 - 1        607.58          101.26         116.16         71.72   
Tyler      4 - 2        622.00          103.67         123.22         87.80   
JT         3 - 3        604.32          100.72         123.16         82.22   
David      3 - 3        561.16           93.53         126.72         71.52   
Jason      2 - 4        647.26          107.88         130.92         86.68   
Kevin      1 - 5        562.30           93.72         123.68         70.84   
Tim        1 - 5        546.22           91.04         115.02         75.14   
Scott      0 - 6        446.38           74.40         102.92         58.22   

        Eyehole Man  Fascist of the Week  
Johnny            1                    0  
Matt              1                    1  
B

## Alternate Records

#### Standings vs All Teams, Alternate Schedules, Standings w/ Median Scoring

### Standings Vs All Teams

This table shows what every team's weekly record and total standings would be if you got points for every team you outscored each week, rather than head to head matchups.

In [17]:
# Step 1: Prepare the Standings vs All Teams DataFrame
standings_vs_all_df = pd.DataFrame(index=played_scores_df.index, columns=[f"Week {i+1}" for i in range(most_recent_week)])

# Step 2: Calculate Wins and Losses for Each Week
# Initialize dictionaries to keep track of total wins and losses across all weeks
total_wins = {manager: 0 for manager in played_scores_df.index}
total_losses = {manager: 0 for manager in played_scores_df.index}

# Loop through each week and calculate standings vs all teams
for week in played_scores_df.columns[:most_recent_week]:
    # Get scores for the current week
    all_weekly_scores = played_scores_df[week]
    
    # Loop through each manager to determine wins and losses
    for manager in played_scores_df.index:
        manager_score = all_weekly_scores[manager]
        
        # Compare against all other managers
        wins = sum(manager_score > other_score for other_score in all_weekly_scores if manager_score > other_score)
        losses = sum(manager_score < other_score for other_score in all_weekly_scores if manager_score < other_score)
        
        # Record the results
        standings_vs_all_df.at[manager, week] = f"{wins} - {losses}"
        
        # Accumulate total wins and losses
        total_wins[manager] += wins
        total_losses[manager] += losses

# Step 3: Add Total Standings Column
standings_vs_all_df["Total Standings"] = [f"{total_wins[manager]} - {total_losses[manager]}" for manager in played_scores_df.index]

# Add a "Total Wins" column for sorting purposes
standings_vs_all_df["Total Wins"] = [total_wins[manager] for manager in played_scores_df.index]

# Step 4: Sort by Total Wins
standings_vs_all_df = standings_vs_all_df.sort_values(by="Total Wins", ascending=False)

# Drop the temporary "Total Wins" column (if you don't want it displayed)
standings_vs_all_df = standings_vs_all_df.drop(columns=["Total Wins"])

# Display the final standings table
print(standings_vs_all_df)

       Week 1 Week 2 Week 3 Week 4 Week 5 Week 6 Total Standings
Jason   3 - 6  8 - 1  9 - 0  6 - 3  3 - 6  8 - 1         37 - 17
Tyler   7 - 2  9 - 0  6 - 3  3 - 6  8 - 1  3 - 6         36 - 18
Johnny  2 - 7  1 - 8  7 - 2  9 - 0  7 - 2  7 - 2         33 - 21
Brian   4 - 5  5 - 4  1 - 8  8 - 1  6 - 3  6 - 3         30 - 24
JT      5 - 4  4 - 5  8 - 1  5 - 4  5 - 4  2 - 7         29 - 25
Matt    9 - 0  0 - 9  3 - 6  7 - 2  4 - 5  5 - 4         28 - 26
Kevin   6 - 3  3 - 6  4 - 5  0 - 9  2 - 7  9 - 0         24 - 30
Tim     8 - 1  7 - 2  2 - 7  4 - 5  1 - 8  1 - 8         23 - 31
David   1 - 8  2 - 7  5 - 4  2 - 7  9 - 0  4 - 5         23 - 31
Scott   0 - 9  6 - 3  0 - 9  1 - 8  0 - 9  0 - 9          7 - 47


### Alternate Schedules

This table shows what each manager's record would be if they had each other manager's schedule. 
When this results in a manager playing with himself (lol), a WIN is recorded if their actual score for that week is greater than their average score for the season. if they score below their average, it goes down as a loss.

These results will be a part of determining strength of schedule, and luck.

In [18]:
# Step 1: Prepare DataFrame for Results
managers = weekly_scores_df.index
simulated_records = pd.DataFrame(index=managers, columns=managers)

# Step 2: Simulate Each Manager's Record Using Every Other Manager's Schedule
for schedule_manager in managers:
    # Initialize a dictionary to store the simulated wins and losses for this schedule
    manager_records = {other_manager: {"wins": 0, "losses": 0} for other_manager in managers}
    
    # Loop through each week's matchups up to the most_recent_week
    for week in weekly_matchups_df.columns[:most_recent_week]:
        # Get the opponent of the schedule manager for this week
        current_opponent = weekly_matchups_df.at[schedule_manager, week]
        
        # If the schedule is not set (None), skip this iteration
        if not current_opponent or current_opponent not in managers:
            continue
        
        # Simulate this schedule for every manager
        for manager in managers:
            # Get the scores for the current `week` using `manager`'s score
            manager_score = weekly_scores_df.at[manager, week]
            
            # Special Case: If the `manager` is playing against themselves
            if manager == current_opponent:
                # Compare against their average score across all played weeks
                played_weeks = weekly_scores_df.loc[manager, :f"Week {most_recent_week}"]
                average_score = played_weeks[played_weeks != 0].mean()  # Exclude unplayed weeks (0)
                
                if manager_score > average_score:
                    manager_records[manager]["wins"] += 1
                else:
                    manager_records[manager]["losses"] += 1
            else:
                # Normal case: Compare `manager`'s score with the actual opponent's score
                opponent_score = weekly_scores_df.at[current_opponent, week]
                
                if manager_score > opponent_score:
                    manager_records[manager]["wins"] += 1
                elif manager_score < opponent_score:
                    manager_records[manager]["losses"] += 1

    # Store the simulated records for this schedule manager
    for manager in managers:
        wins = manager_records[manager]["wins"]
        losses = manager_records[manager]["losses"]
        simulated_records.at[schedule_manager, manager] = f"{wins} - {losses}"

# Step 3: Reorder Both Rows and Columns of `simulated_records` to Match `analysis_df`
ordered_managers = analysis_df.index
simulated_records = simulated_records.reindex(index=ordered_managers, columns=ordered_managers)

# Display the Final Simulated Records Table
print(simulated_records)

       Johnny   Matt  Brian  Tyler     JT  David  Jason  Kevin    Tim  Scott
Johnny  6 - 0  4 - 2  5 - 1  5 - 1  5 - 1  5 - 1  5 - 1  4 - 2  3 - 3  1 - 5
Matt    4 - 2  5 - 1  4 - 2  4 - 2  4 - 2  4 - 2  5 - 1  5 - 1  5 - 1  1 - 5
Brian   4 - 2  4 - 2  5 - 1  4 - 2  4 - 2  3 - 3  5 - 1  4 - 2  2 - 4  1 - 5
Tyler   4 - 2  3 - 3  4 - 2  4 - 2  4 - 2  2 - 4  5 - 1  2 - 4  3 - 3  2 - 4
JT      3 - 3  2 - 4  3 - 3  2 - 4  3 - 3  2 - 4  3 - 3  1 - 5  1 - 5  1 - 5
David   4 - 2  3 - 3  3 - 3  5 - 1  3 - 3  3 - 3  4 - 2  2 - 4  3 - 3  2 - 4
Jason   2 - 4  2 - 4  1 - 5  4 - 2  2 - 4  2 - 4  2 - 4  3 - 3  3 - 3  0 - 6
Kevin   3 - 3  2 - 4  3 - 3  2 - 4  2 - 4  1 - 5  4 - 2  1 - 5  3 - 3  1 - 5
Tim     4 - 2  3 - 3  2 - 4  5 - 1  1 - 5  2 - 4  3 - 3  2 - 4  1 - 5  0 - 6
Scott   3 - 3  3 - 3  3 - 3  4 - 2  3 - 3  1 - 5  4 - 2  2 - 4  2 - 4  0 - 6


In [19]:
# Assuming `weekly_scores_df` and `most_recent_week` are already defined
# Assuming `analysis_df` contains the original wins and losses in a format that can be used.

# Step 1: Initialize Wins and Losses Tally Against Median
wins_vs_median = {manager: 0 for manager in weekly_scores_df.index}
losses_vs_median = {manager: 0 for manager in weekly_scores_df.index}

# Step 2: Calculate Wins and Losses Against Median
for week in weekly_scores_df.columns[:most_recent_week]:
    # Calculate the median score for the current week
    median_score = weekly_scores_df[week].median()
    
    # Compare each manager's score to the median
    for manager in weekly_scores_df.index:
        manager_score = weekly_scores_df.at[manager, week]
        
        if manager_score > median_score:
            wins_vs_median[manager] += 1
        else:
            losses_vs_median[manager] += 1

# Step 3: Create a New DataFrame to Show Results
# Extract the original wins and losses from the analysis_df
original_wins = analysis_df["Standings"].apply(lambda x: int(x.split(" - ")[0]))
original_losses = analysis_df["Standings"].apply(lambda x: int(x.split(" - ")[1]))

# Create the new results DataFrame
results_df = pd.DataFrame(index=weekly_scores_df.index)

# Add Record vs Median
results_df["Record vs Median"] = [
    f"{wins_vs_median[manager]} - {losses_vs_median[manager]}" for manager in weekly_scores_df.index
]

# Calculate combined wins and losses
total_wins = original_wins + pd.Series(wins_vs_median)
total_losses = original_losses + pd.Series(losses_vs_median)

# Add Total Combined Record to the DataFrame
results_df["Total Combined Record"] = [f"{total_wins[manager]} - {total_losses[manager]}" for manager in weekly_scores_df.index]

# Add a temporary column for sorting
results_df["Total Wins"] = total_wins

# Step 4: Sort by Total Wins
results_df = results_df.sort_values(by="Total Wins", ascending=False)

# Drop the temporary "Total Wins" column after sorting
results_df = results_df.drop(columns=["Total Wins"])

# Display the final table
print(results_df)

       Record vs Median Total Combined Record
Johnny            4 - 2                10 - 2
Brian             4 - 2                 9 - 3
Tyler             4 - 2                 8 - 4
Matt              3 - 3                 8 - 4
JT                4 - 2                 7 - 5
Jason             4 - 2                 6 - 6
David             2 - 4                 5 - 7
Tim               2 - 4                 3 - 9
Kevin             2 - 4                 3 - 9
Scott             1 - 5                1 - 11
